In [11]:
!pip install undetected-chromedriver

     ---------------------------------------- 0.0/65.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.4 kB ? eta -:--:--
     ----------------- -------------------- 30.7/65.4 kB 435.7 kB/s eta 0:00:01
     ----------------------------------- -- 61.4/65.4 kB 544.7 kB/s eta 0:00:01
     -------------------------------------- 65.4/65.4 kB 391.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47130 sha256=4f8d5069378e60c08fd5ddde3910fae5b105f4dd5b3151eb621ad36748b4abc6
  Stored in directory: c:\users\dipak\appdata\local\pip\cache\wheels\5c\b9\03\4b6e38f019d6170e8c25df2e1e362d7bdf9ff4012df2dc85c0
Successfully built undetected-chromedriver


In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup


def wait_visible(driver, xpath):
    """Loop until the element becomes visible."""
    while True:
        try:
            el = driver.find_element(By.XPATH, xpath)
            if el.is_displayed():
                return el
        except:
            pass
        time.sleep(0.3)


def scrape_purple_cap(season):

    print(f"\n\n🔵 Scraping IPL {season} – Purple Cap\n")

    driver = webdriver.Chrome()
    driver.maximize_window()
    wait = WebDriverWait(driver, 30)

    driver.get(f"https://www.iplt20.com/stats/{season}")
    time.sleep(5)

    # ACCEPT COOKIES
    try:
        btn = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Accept')]"))
        )
        driver.execute_script("arguments[0].click();", btn)
        print("✔ Cookies accepted")
    except:
        print("⚠ No cookies popup")

    time.sleep(1)

    # CLICK ORANGE CAP DROPDOWN
    print("➡ Opening stats dropdown…")
    dropdown = wait_visible(driver, "(//div[@class='cSBDisplay ng-binding'])[2]")
    driver.execute_script("arguments[0].click();", dropdown)
    time.sleep(1)

    # CLICK BOWLERS
    print("➡ Selecting BOWLERS…")
    bowlers = wait_visible(driver, "//span[contains(text(),'BOWLERS')]")
    driver.execute_script("arguments[0].click();", bowlers)
    time.sleep(1)

    # SCROLL 200px
    driver.execute_script("window.scrollBy(0, 200);")
    time.sleep(1)

    # CLICK PURPLE CAP
    print("➡ Clicking Purple Cap…")
    purple_xpath = "//div[contains(@class,'cSBListItems') and contains(@class,'bowlers') and normalize-space(text())='Purple Cap']"
    purple = wait_visible(driver, purple_xpath)
    driver.execute_script("arguments[0].click();", purple)
    time.sleep(2)

    # SCROLL DOWN FOR VIEW ALL
    print("🔽 Scrolling down to reveal View All…")
    driver.execute_script("window.scrollBy(0, 2000);")
    time.sleep(2)

    # ⭐ MANUAL PART ⭐
    print("\n🟠 CLICK the 'View All' button manually.")
    print("👉 Press ENTER here *after* clicking View All.")
    input("⏳ Waiting for manual click... Press ENTER when done: ")

    # SCRAPE TABLE
    print("📊 Scraping data…")
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table")

    if not table:
        print(f"❌ No table found for {season}")
        driver.quit()
        return None

    rows = table.find("tbody").find_all("tr")
    data = [[td.text.strip() for td in row.find_all("td")] for row in rows]

    columns = [
        "POS","Player","Wkts","Mat","Inns","Ov","Runs",
        "BBI","Avg","Econ","SR","4w","5w"
    ]

    df = pd.DataFrame(data, columns=columns)
    df["Season"] = season

    out = f"purple_cap_{season}.csv"
    df.to_csv(out, index=False)

    print(df)
    print(f"✔ Saved → {out}")

    driver.quit()

    return df



# 🔁 RUN FOR ALL SEASONS
all_seasons_data = []

for season in range(2008, 2026):
    df = scrape_purple_cap(season)
    if df is not None:
        all_seasons_data.append(df)



# COMBINE EVERYTHING INTO ONE CSV
final_df = pd.concat(all_seasons_data, ignore_index=True)
final_df.to_csv("purple_cap_all_seasons.csv", index=False)

print("\n🎉 ALL DONE!")
print("👉 Saved final file: purple_cap_all_seasons.csv")




🔵 Scraping IPL 2008 – Purple Cap

✔ Cookies accepted
➡ Opening stats dropdown…
➡ Selecting BOWLERS…
➡ Clicking Purple Cap…
🔽 Scrolling down to reveal View All…

🟠 CLICK the 'View All' button manually.
👉 Press ENTER here *after* clicking View All.


⏳ Waiting for manual click... Press ENTER when done:  


📊 Scraping data…
     POS                         Player  Wkts   Mat  Inns    Ov  Runs   BBI  \
0   None                           None  None  None  None  None  None  None   
1      1               Sohail Tanvir RR    22    11    11    41   266  6/14   
2      2                 Shane Warne RR    19    15    15    52   404  4/21   
3      3  Shanthakumaran Sreesanth PBKS    19    15    15    51   442  3/29   
4      4             Piyush Chawla PBKS    17    15    15    47   389  4/17   
..   ...                            ...   ...   ...   ...   ...   ...   ...   
83    83             Dinesh Salunkhe RR     1     6     5     8    78  1/21   
84    84          Dwaraka Ravi Teja DEC     1     7     1     2    19  1/19   
85    85            Mohammad Hafeez KKR     1     8     2     5    24   1/8   
86    86              Abhishek Nayar MI     1    14     5     7    68  3/13   
87    87           PM Sarvesh Kumar DEC     1     2     2     5    42  1/18   

      Avg  Econ     SR    4w    5w

⏳ Waiting for manual click... Press ENTER when done:  


📊 Scraping data…
     POS              Player  Wkts   Mat  Inns    Ov  Runs   BBI     Avg  \
0   None                None  None  None  None  None  None  None    None   
1      1        RP Singh DEC    23    16    16    59   417  4/22   18.13   
2      2     Anil Kumble RCB    21    16    16    59   347   5/5   16.52   
3      3    Pragyan Ojha DEC    18    15    15    53   348  3/11   19.33   
4      4   Lasith Malinga MI    18    13    13    49   312  5/13   17.33   
..   ...                 ...   ...   ...   ...   ...   ...   ...     ...   
77    77  Tyron Henderson RR     1     2     2     6    40  1/30   40.00   
78    78    Graham Napier MI     1     1     1     4    27  1/27   27.00   
79    79   Vikram Singh PBKS     1     5     4     9    82  3/29   82.00   
80    80     Chris Gayle KKR     1     7     7    19   144  3/21  144.00   
81    81       Rohan Raje MI     1     4     4     7    72  2/16   72.00   

     Econ      SR    4w    5w  Season  
0    None    None  None  None 

...

📊 Scraping data…
      POS               Player  Wkts   Mat  Inns    Ov  Runs   BBI    Avg  \
0    None                 None  None  None  None  None  None  None   None   
1       1    Mohammad Shami GT    28    17    17  65.0   522  11/4  18.64   
2       2      Mohit Sharma GT    27    14    14  44.1   361  10/5  13.37   
3       3       Rashid Khan GT    27    17    17  67.0   552  30/4  20.44   
4       4     Piyush Chawla MI    22    16    16  61.0   495  22/3  22.50   
..    ...                  ...   ...   ...   ...   ...   ...   ...    ...   
109   109       Akash Deep RCB     1     2     2   5.0    59  29/1  59.00   
110   110  Lockie Ferguson KKR     1     3     3   7.4    96  19/1  96.00   
111   111       Duan Jansen MI     1     1     1   4.0    53  53/1  53.00   
112   112    Shahbaz Ahmed RCB     1    10     5   7.0    95  38/1  95.00   
113   113     Kartik Tyagi SRH     1     3     3   6.1    92  30/1  92.00   

      Econ     SR    4w    5w  Season  
0     None   None 

⏳ Waiting for manual click... Press ENTER when done:  


📊 Scraping data…
      POS                   Player  Wkts   Mat  Inns    Ov  Runs   BBI    Avg  \
0    None                     None  None  None  None  None  None  None   None   
1       1       Harshal Patel PBKS    24    14    14  49.0   477  15/3  19.87   
2       2  Varun Chakaravarthy KKR    21    15    14  50.0   402  16/3  19.14   
3       3        Jasprit Bumrah MI    20    13    13  51.5   336  21/5  16.80   
4       4          T Natarajan SRH    19    14    14  51.2   465  19/4  24.47   
..    ...                      ...   ...   ...   ...   ...   ...   ...    ...   
101   101          Shivam Dube CSK     1    14     1   1.0    14  14/1  14.00   
102   102    Washington Sundar SRH     1     2     2   5.0    73  46/1  73.00   
103   103         Kwena Maphaka MI     1     2     2   6.0    89  23/1  89.00   
104   104             Luke Wood MI     1     2     2   6.0    93  68/1  93.00   
105   105           Akash Deep RCB     1     1     1   3.3    55  55/1  55.00   

      Econ

⏳ Waiting for manual click... Press ENTER when done:  


📊 Scraping data…
      POS                       Player  Wkts   Mat  Inns    Ov  Runs   BBI  \
0    None                         None  None  None  None  None  None  None   
1       1           Prasidh Krishna GT    25    15    15  59.0   488  41/4   
2       2               Noor Ahmad CSK    24    14    14  50.0   408  18/4   
3       3           Josh Hazlewood RCB    22    12    12  44.0   386  33/4   
4       4               Trent Boult MI    22    16    16  57.4   517  26/4   
..    ...                          ...   ...   ...   ...   ...   ...   ...   
104   104          Marcus Stoinis PBKS     1    13     9  14.1   175  14/1   
105   105             Shahrukh Khan GT     1    15     1   1.0    13  13/1   
106   106  V Satyanarayana Penmetsa MI     1     2     2   4.0    53  40/1   
107   107            Musheer Khan PBKS     1     1     1   2.0    27  27/1   
108   108          Mujeeb-ur-Rahman MI     1     1     1   2.0    28  28/1   

        Avg   Econ     SR    4w    5w  Season 

In [47]:
import pandas as pd

files = [
    "purple_cap_2008.csv",
    "purple_cap_2009.csv",
    "purple_cap_2010.csv",
    "purple_cap_2011.csv",
    "purple_cap_2012.csv",
    "purple_cap_2013.csv",
    "purple_cap_2014.csv",
    "purple_cap_2015.csv",
    "purple_cap_2016.csv",
    "purple_cap_2017.csv",
    "purple_cap_2018.csv",
    "purple_cap_2019.csv",
    "purple_cap_2020.csv",
    "purple_cap_2021.csv",
    "purple_cap_2022.csv",
    "purple_cap_2023.csv",
    "purple_cap_2024.csv",
    "purple_cap_2025.csv"
    
]

dfs = [pd.read_csv(f) for f in files]

final_df = pd.concat(dfs, ignore_index=True)
final_df.to_csv("purple_cap_all_seasons.csv", index=False)

print("✔ Combined CSV created successfully!")


✔ Combined CSV created successfully!
